Setting up an VGG16 in Keras. This is quite the more compliated network, in that it goes even deeper, running convolutions over convolutions.

Keras actually has a related repository https://github.com/keras-team/keras-applications/ where you can find a built in, and pretrained VGG16. We'll be implementing it ourselves to get more a sense of how it goes together.

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

Load up the CIFAR images, normalize the images on all color channels 0-1, and one hot encode the labels.

In [2]:
num_outputs = 10 # 10 output digits
batch_size = 128 # mini batch
epochs = 10 # total training loops
learning_rate = 0.01 # amount we update parameters

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = np.expand_dims(train_images / np.max(train_images), -1)
test_images = np.expand_dims(test_images / np.max(test_images), -1)
train_labels = keras.utils.to_categorical(train_labels, num_outputs)
test_labels = keras.utils.to_categorical(test_labels, num_outputs)

VGG16 has 5 blocks, each stacking convolutions. These double in filtering as they grow, which was inspired from older machine vision techniques of 'pyramids'. Think of the layers as forming a kind of step pyramid.

In [4]:
convolution_kernels = [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
convolution_filters = [64, 128, 256, 512, 512]
convolution_repeats = [2, 2, 3, 3, 3]
convolutional_pooling = (2, 2)
dense_units = [4096, 4096]
image_shape = train_images.shape[1:]

This is a loop in a loop, adding convolutional layers end to end before taking a max pooling, looking for the strongest signals. This kind of pyramidal attenuation of filters is a relatively popular approach, and you will see this pattern appear in many different network architectures.

We'll put in one placeholder layer to contain the image shape extracted frome the training data.

Note the use of `same` padding. This actually will pad the images. We need to do this here so that the input image is in fact big enough to 'divide' this many times. You'll see we in the final convolution we end up with a very small x and y dimension.


In [5]:
vgg16 = Sequential()
vgg16.add(Reshape(image_shape[:-1], input_shape=image_shape))
for kernel, filters, repeats in zip(convolution_kernels, convolution_filters, convolution_repeats):
    for _ in range(0, repeats):
        vgg16.add(Conv2D(filters, kernel, activation='relu', padding='same'))
    vgg16.add(MaxPooling2D(convolutional_pooling, padding='same'))
for units in dense_units:
    vgg16.add(Dense(units, activation='relu'))
vgg16.add(Flatten())
vgg16.add(Dense(num_outputs, activation='softmax'))
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
__________

And as always, learning is done with an optimizer and a loss function, learning a classifier with categorical cross entropy.

In [6]:
optimizer = keras.optimizers.SGD(lr=learning_rate)
loss = keras.losses.categorical_crossentropy

Now, keep in mind this is starting to be a pretty big model. If you train this on a CPU, it is *possible*, but it is going to take a very long time. I'm running on a GPU

In [7]:
vgg16.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy'])

history = vgg16.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(test_images, test_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 22s 433us/step - loss: 2.3025 - acc: 0.1019 - val_loss: 2.3024 - val_acc: 0.1000
Epoch 2/10
50000/50000 [==============================] - 14s 286us/step - loss: 2.3024 - acc: 0.1060 - val_loss: 2.3023 - val_acc: 0.1221
Epoch 3/10
50000/50000 [==============================] - 15s 298us/step - loss: 2.3023 - acc: 0.1075 - val_loss: 2.3022 - val_acc: 0.1041
Epoch 4/10
50000/50000 [==============================] - 15s 300us/step - loss: 2.3022 - acc: 0.1113 - val_loss: 2.3021 - val_acc: 0.1099
Epoch 5/10
50000/50000 [==============================] - 15s 296us/step - loss: 2.3020 - acc: 0.1229 - val_loss: 2.3018 - val_acc: 0.1618
Epoch 6/10
50000/50000 [==============================] - 15s 293us/step - loss: 2.3017 - acc: 0.1327 - val_loss: 2.3014 - val_acc: 0.1481
Epoch 7/10
50000/50000 [==============================] - 15s 293us/step - loss: 2.3011 - acc: 0.1413 - val_loss: 2.

Notice that this is a less accurate model than AlexNet for 10 epochs, even though it has less trainable parameters. The network architecture matters, and not all parameters are created equal!

However -- notice that the model is not overfit -- acc and val_acc are very close. We can simply keep training for more epochs until we flatline -- no longer improving the loss and accuracy. And will likely need to!